# 1. Caricamento dataset da google drive

In [ ]:
import os
import shutil
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!unzip /content/drive/MyDrive/dataset_1_componenti/dataset.zip -d /content

In [ ]:
import cv2
import numpy as np
import os
# Mappa gli ID di classe ai nomi di classe
class_labels = {
    0: 'Ceramic',
    1: 'Electrolytic',
    2: 'IC',
    3: 'LED',
    4: 'Resistor',
    5: 'Transistor'
    # Completa con altre classi se necessario
}

def visualize_and_save_polygon_with_limited_labels(image_dir, label_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = [img for img in os.listdir(image_dir) if img.endswith('.jpg') or img.endswith('.png')]

    for img_name in images:
        image_path = os.path.join(image_dir, img_name)
        bbox_path = os.path.join(label_dir, os.path.splitext(img_name)[0] + '.txt')

        image = cv2.imread(image_path)
        ih, iw = image.shape[:2]

        if os.path.exists(bbox_path):
            with open(bbox_path, 'r') as f:
                bboxes = f.readlines()

            # Limita il numero di etichette visualizzate a 5
            for bbox in bboxes[:5]:
                parts = bbox.split(' ')
                class_id = int(parts[0])  # ID di classe
                points = np.array([[float(parts[i]), float(parts[i+1])] for i in range(1, len(parts)-1, 2)])
                points[:, 0] *= iw
                points[:, 1] *= ih
                points = points.astype(np.int32).reshape((-1, 1, 2))

                cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)

                # Ottieni la posizione per disegnare l'etichetta
                label_pos = tuple(points[0][0])
                label_text = class_labels.get(class_id, 'N/A')  # Ottieni il nome della classe

                # Calcola la dimensione del testo per creare uno sfondo
                text_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                text_w, text_h = text_size
                cv2.rectangle(image, label_pos, (label_pos[0] + text_w, label_pos[1] - text_h - 4), (255, 255, 255), -1)

                # Scrivi il testo in blu
                cv2.putText(image, label_text, (label_pos[0], label_pos[1] - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        output_path = os.path.join(output_dir, img_name)
        cv2.imwrite(output_path, image)

image_dir = '/content/dataset/test/images'
label_dir = '/content/dataset/test/labels'
output_dir = '/content/esempi'

visualize_and_save_polygon_with_limited_labels(image_dir, label_dir, output_dir)


In [ ]:
%cd /content
!zip -r esempi.zip esempi/

/content
  adding: esempi/ (stored 0%)
  adding: esempi/images36_jpg.rf.983082e46abccf7debf2c3a59c729abc.jpg (deflated 5%)
  adding: esempi/An3473-kemet-980x655_jpg.rf.8b3aa1cf7709b29213e9836b1b12f46b.jpg (deflated 11%)
  adding: esempi/integrados-condensadores-de-chip-y-las-resistencias-de-chips-Foto-de-archivo_jpg.rf.ed3734518b1f17a3483b334a253e006b.jpg (deflated 3%)
  adding: esempi/Condensadores-electroliticos-de-aluminio_jpg.rf.101e77c8c4e6c8c28dbd8378eb47cad1.jpg (deflated 7%)
  adding: esempi/61D0dNwDfoL-_AC_UF8941000_QL80__jpg.rf.8ce5fbefc978d7ff036f4d7d7f4555e8.jpg (deflated 8%)
  adding: esempi/diodos-led-de-5-mm-para-arduino-varios-colores_jpg.rf.ab5c560247af487b8e5832890a6a2756.jpg (deflated 10%)
  adding: esempi/images27_jpg.rf.f3efd5b1c11b8e0cc3a6e4df6cc2670d.jpg (deflated 24%)
  adding: esempi/Condensadores-electroliticos-Rutronik_jpg.rf.b80e4715a2a092b4611095f72076b5cc.jpg (deflated 7%)
  adding: esempi/images19_jpg.rf.0c7761216f4ecc1fd8d0b97a0adf6ba1.jpg (deflated 16%)

# 2. YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16413, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16413 (delta 0), reused 0 (delta 0), pack-reused 16408
Receiving objects: 100% (16413/16413), 14.96 MiB | 18.15 MiB/s, done.
Resolving deltas: 100% (11263/11263), done.


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
# Copia il file
shutil.copyfile("/content/dataset/data.yaml", "/content/yolov5/dataset.yaml")

'/content/yolov5/dataset.yaml'

# 2.1 Training e validation

In [ ]:
!python train.py --img 416 --batch 16 --epochs 100 --data dataset.yaml --weights yolov5m.pt --cache --name run_50_medium_1

2024-02-06 17:10:27.868956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 17:10:27.869019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 17:10:27.871076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, op

In [ ]:
%cd /content
!zip -r yolov5.zip yolov5/

/content
	zip warning: name not matched: yolov5/

zip error: Nothing to do! (try: zip -r yolov5.zip . -i yolov5/)


# 2.2 Detection

In [ ]:
!python detect.py --source /content/IMG_8006.JPG  --weights runs/train/run_50_medium_1/weights/best.pt --img 416

In [ ]:
!python detect.py --source /content/dataset/test/images  --weights runs/train/run_50_medium_1/weights/best.pt --img 416 --conf 0.1